In [ ]:
import pandas as pd
import ast
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np


In [ ]:
file_path = 'info_data (3).xlsx'
data = pd.read_excel(file_path)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    words = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
    return ' '.join(words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def clean_pub_column(entry):
    if isinstance(entry, str) and entry.startswith('[') and entry.endswith(']'):
        entry = entry[1:-1]
    return entry

In [ ]:
data['pub'] = data['pub'].fillna('').apply(clean_pub_column)

data['pub'] = data['pub'].apply(preprocess_text)

data['Keywords'] = data['Keywords'].fillna('')
data['research_profile'] = data['Keywords'] + ' ' + data['pub']

In [ ]:
print(data['research_profile'][0])

 mannuru n r shahriar teel z wang lund b tijani vaidya p artificial intelligence developing country impact generative artificial intelligence ai technology development information development lund b omame tijani agbaji perception toward artificial intelligence among academic library employee alignment diffusion innovation adopter category college research library lund b agbaji teel z information literacy data literacy privacy literacy chatgpt technology literacy align perspective emerging technology adoption within community lund b agbaji teel za information literacy data literacy privacy literacy chatgpt technology literacy align perspective emerging technology adoption within community human technology lund b agbaji augmented reality browsing physical collection academic library public service quarterly lund b agbaji tijani omame evaluating knowledge organization developed developing country comparative analysis dewey decimal library congress classification scheme preference use unit

In [ ]:
user_input_keywords = " A Text mining-based analysis of Facebook group discussions on gamification"
user_difficulty_category = "Moderate"
user_course = "INFO 5502"

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
research_tfidf = tfidf_vectorizer.fit_transform(data['research_profile'])
user_tfidf = tfidf_vectorizer.transform([user_input_keywords])


In [ ]:
#This code is not working........
#  similarity_scores = cosine_similarity(user_tfidf, research_tfidf).flatten()
# data['similarity_score'] = similarity_scores


# filtered_data = data[(data['course'] == user_course) & (data['Difficulty Category'] == user_difficulty_category)]


# features = filtered_data[['similarity_score', 'Course Difficulty Index']].fillna(0)
# features['similarity_score'] *= 2

# scaler = StandardScaler()
# features_scaled = scaler.fit_transform(features)

# knn_model = NearestNeighbors(n_neighbors=20, metric='euclidean')
# knn_model.fit(features)
# distances, indices = knn_model.kneighbors(features)

# print(distances)

# recommended_professors = filtered_data.iloc[distances[0]]

# print(recommended_professors[['name', 'course', 'similarity_score', 'SFI', 'PEI', 'Course Difficulty Index']])

ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required by StandardScaler.

In [ ]:
# sorting the KNN ouput based on silmilarity score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# Scaling the features as per original code
features = filtered_data[['similarity_score', 'Course Difficulty Index']].fillna(0)
features['similarity_score'] *= 2

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Set up the Nearest Neighbors model
knn_model = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn_model.fit(features_scaled)
distances, indices = knn_model.kneighbors(features_scaled)

# Create a DataFrame for recommendations
recommended_professors = filtered_data.iloc[indices[0]].drop_duplicates(subset='name')

# Sort recommended professors based on SFI for prioritized output
recommended_professors = recommended_professors.sort_values(by=['similarity_score'], ascending=[False])

# Output the final list of recommended professors
print(recommended_professors[['name', 'course', 'similarity_score', 'SFI', 'PEI', 'Course Difficulty Index']])


             name     course  similarity_score   SFI  PEI  \
1747  Junhua Ding  INFO 5502          0.086956  7.62  3.0   

      Course Difficulty Index  
1747                    63.51  


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

user_input_keywords = " A Text mining-based analysis of Facebook group discussions on gamification"
user_difficulty_category = "Easy"
user_course = "INFO 5502"

tfidf_vectorizer = TfidfVectorizer()
research_tfidf = tfidf_vectorizer.fit_transform(data['research_profile'])
user_tfidf = tfidf_vectorizer.transform([user_input_keywords])

similarity_scores = cosine_similarity(user_tfidf, research_tfidf).flatten()
data['similarity_score'] = similarity_scores

# # Check for the presence of the user_course and user_difficulty_category in the data
# print(f"Unique courses in data: {data['course'].unique()}")
# print(f"Unique difficulty categories in data: {data['Difficulty Category'].unique()}")

# Modify filtering to handle potential absence of matching data
filtered_data = data[(data['course'] == user_course) & (data['Difficulty Category'] == user_difficulty_category)]

if filtered_data.empty:
    print("No matching professors found for the given criteria. Please try different criteria.")
else:
    features = filtered_data[['similarity_score', 'Course Difficulty Index']].fillna(0)
    features['similarity_score'] *= 2

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    knn_model = NearestNeighbors(n_neighbors=5, metric='euclidean')
    knn_model.fit(features)
    distances, indices = knn_model.kneighbors(features)

    # Get recommended professors and drop duplicates based on 'name'
    recommended_professors = filtered_data.iloc[indices[0]]
    recommended_professors = recommended_professors.drop_duplicates(subset=['name'])  # Drop duplicate professors

    print(recommended_professors[['name', 'course', 'similarity_score', 'SFI', 'PEI', 'Course Difficulty Index']])

             name     course  similarity_score   SFI  PEI  \
1747  Junhua Ding  INFO 5502          0.086956  7.62  3.0   

      Course Difficulty Index  
1747                    63.51  
